This code produces a polygon from the DWD 1x1km grid.

In [27]:
from netCDF4 import Dataset, MFDataset
import numpy as np
from datetime import datetime, timedelta
from cftime import num2date, date2num, date2index
import geopandas as gpd
from shapely.geometry import Polygon
from climate_data.lat_lon_mask import lat_lon_mask as mask

In [28]:
nc = Dataset('climate_netcdf/zalf_tasmin_amber_2021_v1-0.nc', 'r')

In [29]:
lats = np.array(mask['lat'])
lons = np.array(mask['lon'])

In [30]:
lat_diffs = np.abs(np.diff(lats))
lon_diffs = np.abs(np.diff(lons))

In [31]:
lat_cell_size = np.mean(lat_diffs)
lon_cell_size = np.mean(lon_diffs)

In [32]:
polygons = []
data=[]

In [33]:
for i in range(len(lats)):
    for j in range(len(lons)):
        if mask['mask'][i][j]:
            lat1 = lats[i] + lat_cell_size / 2
            lat2 = lats[i] - lat_cell_size / 2
            lon1 = lons[j] - lon_cell_size / 2
            lon2 = lons[j] + lon_cell_size / 2
            polygons.append(Polygon([(lon1, lat1), (lon2, lat1), (lon2, lat2), (lon1, lat2)]))

            data.append({
            'lat': lats[i],
            'lon': lons[j],
            'lat_idx': i,
            'lon_idx': j
        })
        else:
            continue

In [34]:
gdf = gpd.GeoDataFrame(data, geometry=polygons)
gdf.crs = 'EPSG:4326'  # Set the coordinate reference system to WGS84

# Save to GeoJSON
gdf.to_file('raster_polygons.geojson', driver='GeoJSON')

# Optionally, save to Shapefile
gdf.to_file('raster_polygons.shp')

In [12]:
ws = [{"worksteps": 
       {"date": "2021-03-27",
                    "type": "Sowing",
                    "crop": {
                        "cropParams": {
                            "species": {
                                "=": {
                                    "SpeciesName": "wheat",
                                    "AbovegroundOrgan": [
                                        False,
                                        True,
                                        True,
                                        True
                                    ],
                                    "AssimilateReallocation": 0.3,
                                    "BaseTemperature": [
                                        0,
                                        1,
                                        1,
                                        1,
                                        9,
                                        9
                                    ],
                                    "CarboxylationPathway": 1,
                                    "CriticalOxygenContent": [
                                        0.08,
                                        0.08,
                                        0.08,
                                        0.08,
                                        0.08,
                                        0.08
                                    ],
                                    "CuttingDelayDays": 0,
                                    "DefaultRadiationUseEfficiency": 0.5,
                                    "DevelopmentAccelerationByNitrogenStress": 1.0,
                                    "DroughtImpactOnFertilityFactor": 0.0,
                                    "FieldConditionModifier": 1.0,
                                    "InitialKcFactor": 0.4,
                                    "InitialOrganBiomass": [
                                        53,
                                        53,
                                        0,
                                        0
                                    ],
                                    "InitialRootingDepth": 0.1,
                                    "LimitingTemperatureHeatStress": 40.0,
                                    "LuxuryNCoeff": 1.3,
                                    "MaxCropDiameter": 0.005,
                                    "MaxNUptakeParam": 3.145,
                                    "MinimumNConcentration": 0.005,
                                    "MinimumTemperatureForAssimilation": 0.0,
                                    "MaximumTemperatureForAssimilation": null,
                                    "MinimumTemperatureRootGrowth": 0.0,
                                    "NConcentrationAbovegroundBiomass": 0.06,
                                    "NConcentrationB0": 2.0,
                                    "NConcentrationPN": 1.6,
                                    "NConcentrationRoot": 0.02,
                                    "OrganGrowthRespiration": [
                                        0.306,
                                        0.315,
                                        0.338,
                                        0.291
                                    ],
                                    "OrganMaintenanceRespiration": [
                                        0.01,
                                        0.03,
                                        0.015,
                                        0.01
                                    ],
                                    "PartBiologicalNFixation": 0.0,
                                    "PlantDensity": 220,
                                    "RootDistributionParam": 0.002787,
                                    "RootFormFactor": 3.0,
                                    "RootGrowthLag": -30,
                                    "RootPenetrationRate": 0.0011,
                                    "SamplingDepth": 0.9,
                                    "SpecificRootLength": 300.0,
                                    "StageAfterCut": 0,
                                    "StageAtMaxDiameter": 2,
                                    "StageAtMaxHeight": 3,
                                    "StageMaxRootNConcentration": [
                                        0.02,
                                        0.02,
                                        0.012,
                                        0.01,
                                        0.01,
                                        0.01
                                    ],
                                    "StorageOrgan": [
                                        False,
                                        False,
                                        False,
                                        True
                                    ],
                                    "TargetN30": 120.0,
                                    "TargetNSamplingDepth": 230.0,
                                    "type": "SpeciesParameters"
                                }
                            },
                            "cultivar": {
                                "=": {
                                    "species_name": "wheat",
                                    "AssimilatePartitioningCoeff": [
                                        [
                                            0.5,
                                            0.5,
                                            0,
                                            0
                                        ],
                                        [
                                            0.2,
                                            0.2,
                                            0.6,
                                            0
                                        ],
                                        [
                                            0.13,
                                            0.2,
                                            0.67,
                                            0
                                        ],
                                        [
                                            0,
                                            0,
                                            0.03,
                                            0.97
                                        ],
                                        [
                                            0,
                                            0,
                                            0,
                                            1
                                        ],
                                        [
                                            0,
                                            0,
                                            0,
                                            0.8
                                        ]
                                    ],
                                    "BaseDaylength": [
                                        [
                                            0,
                                            0,
                                            7,
                                            7,
                                            0,
                                            0
                                        ],
                                        "h"
                                    ],
                                    "BeginSensitivePhaseHeatStress": [
                                        780.0,
                                        "\u00b0C d"
                                    ],
                                    "CriticalTemperatureHeatStress": [
                                        31.0,
                                        "\u00b0C"
                                    ],
                                    "CropHeightP1": 6.0,
                                    "CropHeightP2": 0.5,
                                    "CropSpecificMaxRootingDepth": 1.2999999523162842,
                                    "CultivarName": "spring wheat",
                                    "DaylengthRequirement": [
                                        [
                                            0,
                                            20,
                                            20,
                                            20,
                                            0,
                                            0
                                        ],
                                        "h"
                                    ],
                                    "Description": "",
                                    "DroughtStressThreshold": [
                                        1,
                                        0.9,
                                        1,
                                        1,
                                        0.9,
                                        0.8
                                    ],
                                    "EndSensitivePhaseHeatStress": [
                                        900.0,
                                        "\u00b0C d"
                                    ],
                                    "FrostDehardening": 0.0,
                                    "FrostHardening": 0.0,
                                    "HeatSumIrrigationEnd": 1676.0,
                                    "HeatSumIrrigationStart": 461.0,
                                    "LT50cultivar": 0.0,
                                    "LatestHarvestDoy": -1,
                                    "LowTemperatureExposure": 0.0,
                                    "MaxAssimilationRate": 52.0,
                                    "MaxCropHeight": [
                                        0.8299999833106995,
                                        "m"
                                    ],
                                    "OptimumTemperature": [
                                        [
                                            30,
                                            30,
                                            30,
                                            30,
                                            30,
                                            30
                                        ],
                                        "\u00b0C"
                                    ],
                                    "OrganIdsForCutting": [],
                                    "OrganIdsForPrimaryYield": [
                                        {
                                            "type": "YieldComponent",
                                            "organId": 4,
                                            "yieldDryMatter": 0.86,
                                            "yieldPercentage": 0.85
                                        }
                                    ],
                                    "OrganIdsForSecondaryYield": [
                                        {
                                            "type": "YieldComponent",
                                            "organId": 2,
                                            "yieldDryMatter": 0.86,
                                            "yieldPercentage": 0.85
                                        },
                                        {
                                            "type": "YieldComponent",
                                            "organId": 3,
                                            "yieldDryMatter": 0.86,
                                            "yieldPercentage": 0.9
                                        }
                                    ],
                                    "OrganSenescenceRate": [
                                        [
                                            0,
                                            0,
                                            0,
                                            0
                                        ],
                                        [
                                            0,
                                            0,
                                            0,
                                            0
                                        ],
                                        [
                                            0,
                                            0,
                                            0,
                                            0
                                        ],
                                        [
                                            0,
                                            0,
                                            0,
                                            0
                                        ],
                                        [
                                            0,
                                            0.05,
                                            0,
                                            0
                                        ],
                                        [
                                            0,
                                            0.05,
                                            0,
                                            0
                                        ]
                                    ],
                                    "Perennial": False,
                                    "ResidueNRatio": 0.5,
                                    "RespiratoryStress": 0.0,
                                    "SpecificLeafArea": [
                                        [
                                            0.002,
                                            0.0018,
                                            0.0017,
                                            0.0016,
                                            0.0015,
                                            0.0015
                                        ],
                                        "ha kg-1"
                                    ],
                                    "StageKcFactor": [
                                        [
                                            0.6,
                                            0.7,
                                            1.1,
                                            1.1,
                                            0.8,
                                            0.25
                                        ],
                                        "1;0"
                                    ],
                                    "StageTemperatureSum": [
                                        [
                                            80,
                                            284,
                                            280,
                                            180,
                                            435,
                                            25
                                        ],
                                        "\u00b0C d"
                                    ],
                                    "VernalisationRequirement": [
                                        0,
                                        0,
                                        0,
                                        0,
                                        0,
                                        0
                                    ],
                                    "Type": "CultivarParameters"
                                }
                            }
                        },
                        "residueParams": {
                            "AOM_DryMatterContent": [
                                1.0,
                                "kg DM kg FM-1",
                                "Dry matter content of added organic matter"
                            ],
                            "AOM_FastDecCoeffStandard": [
                                0.012,
                                "d-1",
                                "Decomposition rate coefficient of fast AOM at standard conditions"
                            ],
                            "AOM_NH4Content": [
                                0.0002,
                                "kg N kg DM-1",
                                "Ammonium content in added organic matter"
                            ],
                            "AOM_NO3Content": [
                                0.0001,
                                "kg N kg DM-1",
                                "Nitrate content in added organic matter"
                            ],
                            "AOM_SlowDecCoeffStandard": [
                                0.0012,
                                "d-1",
                                "Decomposition rate coefficient of slow AOM at standard conditions"
                            ],
                            "CN_Ratio_AOM_Fast": [
                                0.0,
                                "25",
                                "C to N ratio of the rapidly decomposing AOM pool"
                            ],
                            "CN_Ratio_AOM_Slow": [
                                80.0,
                                "",
                                "C to N ratio of the slowly decomposing AOM pool"
                            ],
                            "NConcentration": [
                                0.005,
                                "kg N kg DM-1",
                                "Nitrogen content in added organic matter"
                            ],
                            "CorgContent": [
                                0.46,
                                "kg C kg DM-1",
                                "Carbon content in added organic matter"
                            ],
                            "PartAOM_Slow_to_SMB_Fast": [
                                0.5,
                                "kg kg-1",
                                "Part of AOM slow consumed by fast soil microbial biomass"
                            ],
                            "PartAOM_Slow_to_SMB_Slow": [
                                0.5,
                                "kg kg-1",
                                "Part of AOM slow consumed by slow soil microbial biomass"
                            ],
                            "PartAOM_to_AOM_Fast": [
                                0.12,
                                "kg kg-1",
                                "Part of AOM that is assigned to the rapidly decomposing pool"
                            ],
                            "PartAOM_to_AOM_Slow": [
                                0.88,
                                "kg kg-1",
                                "Part of AOM that is assigned to the slowly decomposing pool"
                            ],
                            "residueType": "",
                            "species": "wheat",
                            "type": "CropResidueParameters"
                        }
                    }
                },
                {
                    "type": "Harvest",
                    "date": "2021-07-01"
                },
                {
                    "type": "Irrigation",
                    "date": "2021-04-01",
                    "amount": [
                        25.0,
                        "mm"
                    ]
                },
                {
                    "type": "Irrigation",
                    "date": "2021-04-15",
                    "amount": [
                        25.0,
                        "mm"
                    ]
                }
}
            ]
        

SyntaxError: closing parenthesis '}' does not match opening parenthesis '[' on line 1 (237717266.py, line 390)

In [2]:
import json

In [7]:
ws = ws.replace('true', 'True')

In [8]:
ws = ws.replace('false', 'False')

In [11]:
dict = ws.to_dict()


AttributeError: 'str' object has no attribute 'to_dict'